In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
X = dataset.drop(['RowNumber','CustomerId','Surname','Exited'], axis=1)
y = dataset['Exited']
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [4]:
X.shape

(10000, 10)

In [5]:
from sklearn.preprocessing import LabelEncoder
labelencoder_X_1 = LabelEncoder()
X['Geography'] = labelencoder_X_1.fit_transform(X['Geography'])
labelencoder_X_2 = LabelEncoder()
X['Gender'] = labelencoder_X_2.fit_transform(X['Gender'])
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [9]:
classifier = Sequential()
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 10))
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [10]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 7s 926us/step - loss: 0.4844 - accuracy: 0.7955
Epoch 2/100
8000/8000 [==============================] - 6s 797us/step - loss: 0.4355 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 6s 745us/step - loss: 0.4332 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 6s 731us/step - loss: 0.4290 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 8s 965us/step - loss: 0.4241 - accuracy: 0.8136
Epoch 6/100
8000/8000 [==============================] - 7s 864us/step - loss: 0.4210 - accuracy: 0.8245
Epoch 7/100
8000/8000 [==============================] - 6s 742us/step - loss: 0.4189 - accuracy: 0.8286
Epoch 8/100
8000/8000 [==============================] - 5s 610us/step - loss: 0.4168 - accuracy: 0.8290
Epoch 9/100
8000/8000 [==============================] - 3s 428us/step - loss: 0.4154 - accuracy: 0.8296
Epoch 10/100
8000/8000 [==============================]

In [11]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [12]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1543,   52],
       [ 269,  136]])

In [14]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [15]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 10))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [16]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size=10, nb_epoch=100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv=10, n_jobs=-1)

In [17]:
accuracies.mean()

0.7960000097751617

In [18]:
accuracies

array([0.78625   , 0.79000002, 0.80000001, 0.78250003, 0.81625003,
       0.81      , 0.78750002, 0.79374999, 0.79874998, 0.79500002])

In [19]:
accuracies.std()

0.010105690527913859